In [3]:
import os
import numpy as np
from lsst.daf.butler import Butler
import lsst.geom as geom

def get_one_deep_coadd_ref(butler, ra, dec, band, datasetType="deep_coadd"):
    refs = list(butler.query_datasets(
        datasetType,
        where="band.name = band AND patch.region OVERLAPS POINT(ra, dec)",
        bind={"band": band, "ra": ra, "dec": dec},
        with_dimension_records=True,
        order_by=["patch.tract"],
    ))
    if not refs:
        raise RuntimeError(f"No {datasetType} found for band={band} at ra,dec={ra},{dec}")
    return refs[0]

def load_patch_exposures(
    ra, dec,
    bands=("g","r","i"),
    repo="dp1",
    collection="LSSTComCam/DP1",
    datasetType="deep_coadd",
):
    butler = Butler(repo, collections=collection)

    ref0 = get_one_deep_coadd_ref(butler, ra, dec, bands[0], datasetType=datasetType)

    # Convert DataCoordinate -> plain python dict safely
    base_dataId = dict(ref0.dataId.mapping)


    exps = {}
    for b in bands:
        dataId = dict(base_dataId)          # normal dict copy
        dataId["band"] = b                  # override just the band
        exps[b] = butler.get(datasetType, dataId=dataId)

    wcs_full = exps[bands[0]].getWcs()
    return exps, wcs_full, base_dataId


def tile_patch_and_save(
    exps, wcs_full,
    out_dir,
    tile_size=512,
    stride=256,
    bands=("g","r","i"),
    max_tiles=None,
):
    os.makedirs(out_dir, exist_ok=True)

    # assume all bands share same array shape
    H, W = exps[bands[0]].image.array.shape
    n_saved = 0

    # iterate tile upper-left corners
    for y0 in range(0, H - tile_size + 1, stride):
        for x0 in range(0, W - tile_size + 1, stride):
            bbox = geom.BoxI(geom.PointI(x0, y0), geom.ExtentI(tile_size, tile_size))
            # cutout-local WCS: pixel origin shifted so (0,0) corresponds to (x0,y0) in full patch
            wcs_local = wcs_full.copyAtShiftedPixelOrigin(geom.Extent2D(-x0, -y0))

            imgs = []
            vars_ = []
            masks = []
            for b in bands:
                exp = exps[b]

                # slice arrays directly (fast)
                img = exp.image.array[y0:y0+tile_size, x0:x0+tile_size].astype(np.float32)
                var = exp.variance.array[y0:y0+tile_size, x0:x0+tile_size].astype(np.float32)
                msk = exp.mask.array[y0:y0+tile_size, x0:x0+tile_size].astype(np.int32)

                imgs.append(img)
                vars_.append(var)
                masks.append(msk)

            imgs = np.stack(imgs, axis=0)   # [B,H,W]
            vars_ = np.stack(vars_, axis=0) # [B,H,W]
            masks = np.stack(masks, axis=0) # [B,H,W]

            # minimal WCS serialization: store CD + CRPIX + CRVAL from the local WCS
            # (enough to reconstruct later; you can expand this if needed)
            md = wcs_local.getFitsMetadata()
            wcs_hdr = {k: md.getScalar(k) for k in md.names()}

            fn = os.path.join(out_dir, f"tile_x{x0:05d}_y{y0:05d}.npz")
            np.savez_compressed(
                fn,
                img=imgs,
                var=vars_,
                mask=masks,
                wcs_hdr=wcs_hdr,
                x0=np.int32(x0),
                y0=np.int32(y0),
            )
            n_saved += 1
            if (max_tiles is not None) and (n_saved >= max_tiles):
                return n_saved
    return n_saved

# ---- Example: ECDFS center (rough) ----
ra_ecdfs  = 53.16   # deg (rough)
dec_ecdfs = -28.10    # deg (rough)

bands = ("g","r","i")  # start small; add u,z later
exps, wcs_full, dataId0 = load_patch_exposures(
    ra_ecdfs, dec_ecdfs, bands=bands,
    repo="dp1", collection="LSSTComCam/DP1",
    datasetType="deep_coadd",
)

print("Using patch:", dataId0)

n = tile_patch_and_save(
    exps, wcs_full,
    out_dir="rubin_tiles_ecdfs_gr_i",
    tile_size=512,
    stride=256,
    bands=bands,
    max_tiles=200,   # just to start
)
print("Saved tiles:", n)


Using patch: {'band': 'g', 'skymap': 'lsst_cells_v1', 'tract': 5063, 'patch': 14}
Saved tiles: 144
